In [4]:
import os
import pydicom
import pandas as pd
import numpy as np
from datetime import datetime
import shutil

In [2]:
# Identify DICOM files under 'DICOMpath' directory
DICOMpath='C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2'
DICOMlist = []
for (path, dir, files) in os.walk(DICOMpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            DICOMlist.append(filePath)
Metadata=list(map(lambda x : pydicom.filereader.dcmread(x), DICOMlist))

C:\Users\ABMI\Anaconda3\lib\site-packages\pydicom\charset.py:658: UserWarning: Incorrect value for Specific Character Set 'ISO IR 6' - assuming 'ISO_IR 6'
  _warn_about_invalid_encoding(encoding, patched)


In [3]:
def ColumnsFunction(x):
    try:
        return Metadata[x].Columns
    except:
        return 'NA'

In [4]:
def RowsFunction(x):
    try:
        return Metadata[x].Rows
    except:
        return 'NA'

In [5]:
def SeriesInstanceFunction(x):
    try:
        return Metadata[x].SeriesInstanceUID
    except:
        return 'NA'

In [6]:
def SOPInstanceUIDFunction(x):
    try:
        return Metadata[x].SOPInstanceUID
    except:
        return 'NA'

In [7]:
def StudyInstanceUIDFunction(x):
    try:
        return Metadata[x].StudyInstanceUID
    except:
        return 'NA'

In [8]:
def PatientIDFunction(x):
    try:
        return Metadata[x].PatientID
    except:
        return 'NA'

In [9]:
def SliceThicknessFunction(x):
    try:
        return Metadata[x].SliceThickness
    except:
        return 'NA'

In [10]:
#Create RadiologyImageTable without columns of ImageOrientation, RadiologyPhaseConceptID
RadiologyImageTable=pd.DataFrame({'Filepath': DICOMlist, 'ImageID':list(map(lambda x : SOPInstanceUIDFunction(x), range(len(DICOMlist)))), 'SeriesID':list(map(lambda x : SeriesInstanceFunction(x), range(len(DICOMlist)))), 'RadiologyOccurrenceID':list(map(lambda x : StudyInstanceUIDFunction(x), range(len(DICOMlist)))), 'PatientID':list(map(lambda x : PatientIDFunction(x), range(len(DICOMlist)))), 'ImageResolutionRows':list(map(lambda x : RowsFunction(x), range(len(DICOMlist)))), 'ImageResolutionColumns':list(map(lambda x : ColumnsFunction(x), range(len(DICOMlist)))), 'SliceThickness':list(map(lambda x : SliceThicknessFunction(x), range(len(DICOMlist))))})
grouped=RadiologyImageTable[['ImageID','SeriesID']].groupby('SeriesID')
PhaseTotalNo=grouped.count()
PhaseTotalNo.columns = ['PhaseTotalNo']
RadiologyImageTable['ImageNo']=RadiologyImageTable.groupby(['SeriesID']).cumcount()+1
RadiologyImageTable=pd.merge(RadiologyImageTable, PhaseTotalNo, on='SeriesID')

In [11]:
RadiologyImageTable

,Filepath,ImageID,SeriesID,RadiologyOccurrenceID,PatientID,ImageResolutionRows,ImageResolutionColumns,SliceThickness,ImageNo,PhaseTotalNo
0,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,9.1.48564849485154484851.1.1182212.3,9.1.48564849485154484851.1,9.1.48564849485154484851,0710537,426,568,NA,1,2
1,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,9.1.48564849485154484851.1.1182220.4,9.1.48564849485154484851.1,9.1.48564849485154484851,0710537,426,568,NA,2,2
2,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,9.1.48564849485254495754.1.1183323.2,9.1.48564849485254495754.1,9.1.48564849485254495754,0931596,426,568,NA,1,2
3,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,9.1.48564849485254495754.1.1183328.3,9.1.48564849485254495754.1,9.1.48564849485254495754,0931596,426,568,NA,2,2
4,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,1.3.12.2.1107.5.1.4.50373.30000008010421505873...,1.3.12.2.1107.5.1.4.50373.30000008010421505873...,1.2.410.2000010.82.220.10801070004,0956645,512,512,4.5,1,13
...,...,...,...,...,...,...,...,...,...,...
2575,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\LIDC...,1.3.6.1.4.1.14519.5.2.1.6279.6001.245155194022...,1.3.6.1.4.1.14519.5.2.1.6279.6001.154696925868...,1.3.6.1.4.1.14519.5.2.1.6279.6001.176996323847...,LIDC-IDRI-0029,2758,2465,NA,1,1
2576,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\LIDC...,1.3.6.1.4.1.14519.5.2.1.6279.6001.976782320365...,1.3.6.1.4.1.14519.5.2.1.6279.6001.115231804715...,1.3.6.1.4.1.14519.5.2.1.6279.6001.256178497642...,LIDC-IDRI-0030,2022,1736,NA,1,4
2577,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\LIDC...,1.3.6.1.4.1.14519.5.2.1.6279.6001.185557258541...,1.3.6.1.4.1.14519.5.2.1.6279.6001.115231804715...,1.3.6.1.4.1.14519.5.2.1.6279.6001.256178497642...,LIDC-IDRI-0030,2022,1736,NA,2,4
2578,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\LIDC...,1.3.6.1.4.1.14519.5.2.1.6279.6001.640511412671...,1.3.6.1.4.1.14519.5.2.1.6279.6001.115231804715...,1.3.6.1.4.1.14519.5.2.1.6279.6001.256178497642...,LIDC-IDRI-0030,2022,1736,NA,3,4


In [12]:
def StudyDateFunction(x):
    try:
        return Metadata[x].StudyDate
    except:
        return ''
def StudyTimeFunction(x):
    try:
        return Metadata[x].StudyTime[0:6]
    except:
        return ''
    
StudyDate=list(map(lambda x : StudyDateFunction(x), range(len(DICOMlist))))
StudyTime=list(map(lambda x : StudyTimeFunction(x), range(len(DICOMlist))))

def StudyDateTime(x):
    DateTime=StudyDate[x]+StudyTime[x]
    try:
        return str(datetime.strptime(DateTime, '%Y%m%d%H%M%S'))
    except:
        try:
            return str(datetime.strptime(DateTime, '%Y%m%d'))
        except:
            return str(datetime.strptime(DateTime, ''))

In [57]:
def Manufacturer(x):
    try:
        return Metadata[x].Manufacturer
    except:
        return 'NA'

In [58]:
def Modality(x):
    try:
        return Metadata[x].Modality
    except:
        return 'NA'

In [59]:
RadiologyOccurrenceTable=pd.DataFrame({'RadiologyOccurrenceID':list(map(lambda x : StudyInstanceUIDFunction(x), range(len(DICOMlist)))), 'StudyDateTime':list(map(lambda x : StudyDateTime(x), range(len(DICOMlist)))), 'PatientID':list(map(lambda x : PatientIDFunction(x), range(len(DICOMlist)))), 'Manufacturer':list(map(lambda x : Manufacturer(x), range(len(DICOMlist)))), 'Modality':list(map(lambda x : Modality(x), range(len(DICOMlist))))})

In [74]:
RadiologyOccurrenceTable.groupby('RadiologyOccurrenceID').first().reset_index()

,RadiologyOccurrenceID,StudyDateTime,PatientID,Manufacturer,Modality
0,9.1.48564849485154484851,2008-01-07 18:22:12,0710537,DATA HUB,AP
1,9.1.48564849485154484851,2008-01-07 18:22:20,0710537,DATA HUB,AP
2,9.1.48564849485254495754,2008-01-07 18:33:23,0931596,DATA HUB,AP
3,9.1.48564849485254495754,2008-01-07 18:33:28,0931596,DATA HUB,AP
4,1.2.410.2000010.82.220.10801070004,2008-01-07 00:18:41,0956645,SIEMENS,CT
...,...,...,...,...,...
2575,1.3.6.1.4.1.14519.5.2.1.6279.6001.176996323847...,2000-01-01 00:00:00,LIDC-IDRI-0029,Philips Medical Systems,CR
2576,1.3.6.1.4.1.14519.5.2.1.6279.6001.256178497642...,2000-01-01 00:00:00,LIDC-IDRI-0030,GE MEDICAL SYSTEMS,DX
2577,1.3.6.1.4.1.14519.5.2.1.6279.6001.256178497642...,2000-01-01 00:00:00,LIDC-IDRI-0030,GE MEDICAL SYSTEMS,DX
2578,1.3.6.1.4.1.14519.5.2.1.6279.6001.256178497642...,2000-01-01 00:00:00,LIDC-IDRI-0030,GE MEDICAL SYSTEMS,DX


In [89]:
### Image classification by SeriesDescription(Brain CT / Brain MR - T1, T2, FLAIR / Abd. CT - non, arterial, portal, delay / Chest XR - AP, PA / others)
# Abd.CT - non : Pre 5.0 B35f, Pre 5.0 B45f
# Abd.CT - arterial : Artery 3.0 B35f, Artery phase 5.0 B45f
# Abd.CT - portal : Portal 5.0 B35f, Portal phase 5.0 B45f
# Abd.CT - delay : DELAY 5.0 B35f, Delay phase 5.0 B45f

# BodyPartExamined, ProtocolName, StudyDescription
# Brain MR - T1 axial : AX T1, T1/AX, AXL T1 SE, AXL SE T1
# Brain MR - T1 sagittal : SA T1, T1/SAG, SAG T1 SE, SAG SE T1
# Brain MR - T2 axial : AX T2, T2/AX, AXL FSE T2, T2 TSE AX, AXL T2 FRFSE
# Brain MR - T2 coronal : T2/COR-OBL, COR T2
# Brain MR - FLAIR axial : AX FLAIR, T2/FLAIR/AX, AXL FLAIR, T2 FLAIR AX, AXL T2 FLAIR, FLAIR
# Brain MR - FLAIR coronal : T2/FLAIR/COR-OBL
# Brain MR - ADC : Apparent Diffusion Coefficient
DICOMpath='D:/abd_CT'
abdCTnon=['Pre  5.0  B35f', 'Pre  5.0  B45f']
abdCTarterial=['Artery  3.0  B35f', 'Artery phase  5.0  B45f']
abdCTportal=['Portal   5.0  B35f', 'Portal phase  5.0  B45f']
abdCTdelay=['DELAY  5.0  B35f', 'Delay phase  5.0  B45f']
brainMRT1axial=['AX T1', 'T1/AX', 'AXL T1 SE', 'AXL SE T1']
brainMRT1sag=['SA T1', 'T1/SAG', 'SAG T1 SE', 'SAG SE T1']
brainMRT2axial=['AX T2', 'T2/AX', 'AXL FSE T2', 'T2 TSE AX', 'AXL T2 FRFSE']
brainMRT2cor=['T2/COR-OBL', 'COR T2']
brainMRFLAIRaxial=['AX FLAIR', 'T2/FLAIR/AX', 'AXL FLAIR', 'T2 FLAIR AX', 'AXL T2 FLAIR', 'FLAIR']
brainMRFLAIRcor=['T2/FLAIR/COR-OBL']
brainMRADC=['Apparent Diffusion Coefficient']
errorfile=[]
for (path, dir, files) in os.walk(DICOMpath):
    for filename in files:
        try:
            ext = os.path.splitext(filename)[-1]
            if ext == '.dcm':
                filePath = "{0}/{1}".format(path, filename)
                metadata=pydicom.filereader.dcmread(filePath)
                try:
                    if metadata.Modality=='CT' and metadata.BodyPartExamined.lower()=='abdomen' and metadata.SeriesDescription in abdCTnon:
                        shutil.copy(filePath, 'D:/class/abdCTnon')
                    elif metadata.Modality=='CT' and metadata.BodyPartExamined.lower()=='abdomen' and metadata.SeriesDescription in abdCTarterial:
                        shutil.copy(filePath, 'D:/class/abdCTarterial')
                    elif metadata.Modality=='CT' and metadata.BodyPartExamined.lower()=='abdomen' and metadata.SeriesDescription in abdCTportal:
                        shutil.copy(filePath, 'D:/class/abdCTportal')
                    elif metadata.Modality=='CT' and metadata.BodyPartExamined.lower()=='abdomen' and metadata.SeriesDescription in abdCTdelay:
                        shutil.copy(filePath, 'D:/class/abdCTdelay')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRT1axial and True in ['brain' in i for i in [metadata.BodyPartExamined.lower(), metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRT1axial')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRT1sag and True in ['brain' in i for i in [metadata.BodyPartExamined.lower(), metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRT1sag')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRT2axial and True in ['brain' in i for i in [metadata.BodyPartExamined.lower(), metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRT2axial')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRT2cor and True in ['brain' in i for i in [metadata.BodyPartExamined.lower(), metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRT2cor')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRFLAIRaxial and True in ['brain' in i for i in [metadata.BodyPartExamined.lower(), metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRFLAIRaxial')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRFLAIRcor and True in ['brain' in i for i in [metadata.BodyPartExamined.lower(), metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRFLAIRcor')
                    elif metadata.Modality=='MR' and 'Apparent Diffusion Coefficient' in metadata.SeriesDescription and True in ['brain' in i for i in [metadata.BodyPartExamined.lower(), metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRADC')
                except:
                    if metadata.Modality=='MR' and metadata.SeriesDescription in brainMRT1axial and True in ['brain' in i for i in [metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRT1axial')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRT1sag and True in ['brain' in i for i in [metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRT1sag')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRT2axial and True in ['brain' in i for i in [metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRT2axial')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRT2cor and True in ['brain' in i for i in [metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRT2cor')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRFLAIRaxial and True in ['brain' in i for i in [metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRFLAIRaxial')
                    elif metadata.Modality=='MR' and metadata.SeriesDescription in brainMRFLAIRcor and True in ['brain' in i for i in [metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRFLAIRcor')
                    elif metadata.Modality=='MR' and 'Apparent Diffusion Coefficient' in metadata.SeriesDescription and True in ['brain' in i for i in [metadata.ProtocolName.lower(), metadata.StudyDescription.lower()]]:
                        shutil.copy(filePath, 'D:/class/brainMRADC')
        except:
            errorfile.append(filePath)